In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 34.5 MB/s 
     |████████████████████████████████| 182 kB 77.2 MB/s 
     |████████████████████████████████| 7.6 MB 83.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 25.3 MB/s 
     |████████████████████████████████| 132 kB 75.6 MB/s 
     |████████████████████████████████| 212 kB 84.5 MB/s 
     |████████████████████████████████| 127 kB 95.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 37.1 MB/s 


In [ ]:
#import libraries
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm
from sklearn.utils import shuffle
from google.colab import drive
from tqdm import tqdm
import regex as re
import string
import nltk
import pickle
import datasets
from datasets import Dataset
import torch



import transformers
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel, EncoderDecoderModel, PegasusForConditionalGeneration, PegasusTokenizer
from tqdm import tqdm_notebook as tqdm


In [ ]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#load the scrapped data
df=pd.read_csv("/content/drive/MyDrive/NLP_Project/final_imdb_movies.csv")

In [ ]:
df.head()

,Name,link,summary,storyline
0,The Shawshank Redemption,/title/tt0111161/,Two imprisoned men bond over a number of years...,Chronicles the experiences of a formerly succe...
1,The Godfather,/title/tt0068646/,The aging patriarch of an organized crime dyna...,"The Godfather ""Don"" Vito Corleone is the head ..."
2,Pulp Fiction,/title/tt0110912/,"The lives of two mob hitmen, a boxer, a gangst...",Jules Winnfield (Samuel L. Jackson) and Vincen...
3,The Godfather Part II,/title/tt0071562/,The early life and career of Vito Corleone in ...,The continuing saga of the Corleone crime fami...
4,Inception,/title/tt1375666/,A thief who steals corporate secrets through t...,"Dom Cobb is a skilled thief, the absolute best..."


In [ ]:
print(df["link"].nunique(), df["link"].count())

786 787


In [ ]:
df['link'].value_counts()

/title/tt0406759/    2
/title/tt0111161/    1
/title/tt0278488/    1
/title/tt0089880/    1
/title/tt0070034/    1
                    ..
/title/tt0449467/    1
/title/tt1038988/    1
/title/tt0346491/    1
/title/tt0277371/    1
/title/tt1073105/    1
Name: link, Length: 786, dtype: int64

In [ ]:
#drop any duplicates
df.drop_duplicates(inplace=True)

In [ ]:
print(df["link"].nunique(), df["link"].count())

786 786


In [ ]:
#load punctuations
punctuations = set(string.punctuation)

In [ ]:
# remove special characters, digits, html tags ans punctuations
def preprocess(text):
    text = text.lower() #converting capitals to lower case
    text = text.strip()
    text= re.sub(r'\—.*$', " ", text)
    text=re.sub(r'\([^)]*\)', '', text)
    text = re.sub("<.*?>", " ", text) #removing html tags
    text = re.sub(r'[^\w\s]',' ',text) #remove special characters
    text = re.sub(" \d+", " ",text) # removing digits
    text = re.sub("\n", "", text) #removing \n
    text = re.sub("\t", "", text) #removing \t
    text = ''.join(ch for ch in text if ch not in punctuations)
    return text

In [ ]:
df["summary"] = df["summary"].apply(preprocess)

In [ ]:
df["storyline"] = df["storyline"].apply(preprocess)

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments

tokenizer_pretrained = PegasusTokenizer.from_pretrained('google/pegasus-xsum')


Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

In [ ]:
nltk.download('stopwords')

from nltk.tokenize.toktok import ToktokTokenizer
space_tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

#removing stopwords
stop=set(stopwords.words('english')) #setting language to english

def remove_stopwords(text, is_lower_case=False):
    tokens = space_tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

df['summary']=df['summary'].apply(remove_stopwords)

df["storyline"]= df["storyline"].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#split data
from sklearn.model_selection import train_test_split

train, test = train_test_split(df,random_state=10, test_size=0.2)

# Create Enbeddings

In [ ]:
# create embeddings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
#hyper-parameters

batch_size = 8
encoder_max_length = 512
decoder_max_length = 64


In [ ]:
# encoder using - bert_base_uncased
def bert_base_uncased_encoder(batch):
  storyline_encoded = tokenizer(batch["storyline"], padding="max_length", truncation=True, max_length=encoder_max_length)
  summary_encoded = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)
  batch["input_ids"]= storyline_encoded.input_ids
  batch["attention_mask"]= storyline_encoded.attention_mask
  batch["decoder_input_ids"]=summary_encoded.input_ids
  batch["decoder_attention_mask"]=summary_encoded.attention_mask
  batch["labels"] = summary_encoded.input_ids.copy()
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch


In [ ]:
# convert change df to huggingface Dataset
train_data = Dataset.from_pandas(train)
test_data = Dataset.from_pandas(test)

In [ ]:
#embeddings on training data
#Note: If data has already been mapped and saved to file DO NOT run this cell

# Uncomment out the following two line two train on a small subset
# train_data = train_data.select(range(32))
# val_data = val_data.select(range(32))

remove_cols = ["storyline","summary","link","Name"]
required_cols = ["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"]
train_data = train_data.map(
    bert_base_uncased_encoder,
    batched=True,
    batch_size=batch_size,
    remove_columns=remove_cols
)

train_data.set_format(
    type="torch", columns=required_cols
)

#embeddings on validation data
# val_data = val_data.map(
#     bert_base_uncased_encoder,
#     batched = True,
#     batch_size = batch_size,
#     remove_columns = remove_cols
# )
# val_data.set_format(
#     type="torch", columns = required_cols
# )

  0%|          | 0/79 [00:00<?, ?ba/s]

In [ ]:
#save_mapped_data

with open('bert_base_uncased_train_data.pickle', 'wb') as handle:
  pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


# with open('bert_base_uncased_val_data.pickle', 'wb') as handle:
#     pickle.dump(val_data, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
#Creates the base encoder decoder model
model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased","bert-base-uncased")

# set model configuration
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
model.config.vocab_size = model.config.decoder.vocab_size
model.config.max_length = 128
model.config.min_length = 64
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#compute rouge score
import datasets
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output1 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
    rouge_output2 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge1_precision": round(rouge_output1.precision, 4),
        "rouge1_recall": round(rouge_output1.recall, 4),
        "rouge1_fmeasure": round(rouge_output1.fmeasure, 4),
        "rouge2_precision": round(rouge_output2.precision, 4),
        "rouge2_recall": round(rouge_output2.recall, 4),
        "rouge2_fmeasure": round(rouge_output2.fmeasure, 4),
    }

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
#trainer 
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/NLP_Project/Seq2Seq/results", # output directory
    per_device_train_batch_size=1,  # batch size per device during training, can increase if memory allows
    predict_with_generate=True,
    do_train=True,
    save_steps=628,   # number of updates steps before checkpoint saves
    warmup_steps=50,  # number of warmup steps for learning rate scheduler
    overwrite_output_dir=True,
    save_total_limit=3,  # limit the total amount of checkpoints and deletes the older checkpoints
    fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 628
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 1884
  Number of trainable parameters = 247363386


Step,Training Loss
500,0.121500
1000,0.003400
1500,0.000200


Saving model checkpoint to /content/drive/MyDrive/NLP_Project/Seq2Seq/results/checkpoint-628
Configuration saved in /content/drive/MyDrive/NLP_Project/Seq2Seq/results/checkpoint-628/config.json
Model weights saved in /content/drive/MyDrive/NLP_Project/Seq2Seq/results/checkpoint-628/pytorch_model.bin
/usr/local/lib/python3.8/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to /content/drive/MyDrive/NLP_Project/Seq2Seq/results/checkpoint-1256
Configuration saved in /content/drive/MyDrive/NLP_Project/Seq2

TrainOutput(global_step=1884, training_loss=0.03323804381285693, metrics={'train_runtime': 479.7091, 'train_samples_per_second': 3.927, 'train_steps_per_second': 3.927, 'total_flos': 1155747525304320.0, 'train_loss': 0.03323804381285693, 'epoch': 3.0})

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

26

In [ ]:
#load the pre-trained model
model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/NLP_Project/Seq2Seq/results/checkpoint-628/")
# model.to("cuda") # Comment out this line if not using CUDA
batch_size = 15

def evaluate_test_data(batch):
    inputs = tokenizer(batch["storyline"], padding="max_length", truncation=True, max_length = decoder_max_length, return_tensors="pt")
    input_ids = inputs.input_ids # Remove ".to("cuda")" if not using CUDA
    attention_mask = inputs.attention_mask # Remove ".to("cuda")" if not using CUDA
    decoder_input_ids = inputs.input_ids
    
    outputs = model.generate(input_ids, attention_mask=attention_mask, decoder_start_token_id=decoder_input_ids, max_new_tokens = 10)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch


results = test_data.map(evaluate_test_data, batched=True, batch_size=batch_size, remove_columns=["storyline"])

pred_str = results["pred"]
label_str = results["summary"]

rouge_output1 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
rouge_output2 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output1)
print(rouge_output2)

  0%|          | 0/11 [00:00<?, ?ba/s]

NameError: ignored

In [ ]:
test

,Name,link,summary,storyline
237,8 Mile,/title/tt0298203/,young rapper struggling every aspect life want...,setting detroit city divided mile road splits ...
491,Shark Tale,/title/tt0307453/,son gangster shark boss accidentally killed hu...,sea underworld shaken son shark mob boss lino ...
68,Batman Begins,/title/tt0372784/,training mentor batman begins fight free crime...,parents killed billionaire playboy bruce wayne...
740,Forrest Gump,/title/tt0109830/,presidencies kennedy johnson vietnam war water...,forrest gump simple man low q good intentions ...
458,Mission: Impossible II,/title/tt0120755/,imf agent ethan hunt sent sydney find destroy ...,chimera deadly virus bear grisly death unless ...
...,...,...,...,...
193,Spider-Man 2,/title/tt0316654/,peter parker beset troubles failing personal l...,peter parker unhappy man two years fighting cr...
194,The Butterfly Effect,/title/tt0289879/,evan treborn suffers blackouts significant eve...,child evan treborn afflicted blackouts would o...
610,Shaft,/title/tt0162650/,new york city police detective john shaft goes...,new york police detective john shaft lead dete...
383,The Sweetest Thing,/title/tt0253867/,christina walters spent years avoiding men sud...,christina walters courtney rockcliffe take roo...


In [ ]:
tokens = tokenizer(test['storyline'][237], truncation=True, padding="longest",max_length = decoder_max_length, return_tensors="pt")
input_ids = tokens.input_ids # Remove ".to("cuda")" if not using CUDA
attention_mask = tokens.attention_mask # Remove ".to("cuda")" if not using CUDA
decoder_input_ids = tokens.input_ids
summary_ = model.generate(input_ids, attention_mask=attention_mask, decoder_start_token_id=decoder_input_ids,  max_new_tokens = 30)
tokenizer.decode(summary_[0])

'[CLS] setting detroit city divided mile road splits town half along racial lines young white rapper jimmy b rabbit smith jr summons strength within cross arbitrary boundaries fulfill dream success hip hop pal future three one third place choke [SEP]...???!!! ！ ！ ！ ： ： ： ！ ！ ¡ ¡ ¡ ！ ！ 、 ！ ！ ？ ！ ！! ！'

In [ ]:
test['storyline'][237]

'setting detroit city divided mile road splits town half along racial lines young white rapper jimmy b rabbit smith jr summons strength within cross arbitrary boundaries fulfill dream success hip hop pal future three one third place choke'

In [ ]:
test['summary'][237]

'young rapper struggling every aspect life wants make big friends foes make odyssey rap harder may seem'

In [ ]:
!pip install sumeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 80 kB 8.1 MB/s 
     |████████████████████████████████| 118 kB 85.6 MB/s 
  Created wheel for sumeval: filename=sumeval-0.2.2-py3-none-any.whl size=54549 sha256=dad0d8b971b95a8a422b64e1766b9314b4695c0d0fbcb0804a44999ce0185c9c
  Stored in directory: /root/.cache/pip/wheels/6a/e8/4e/76111a2e023408af67380b35a6a910763432fc0afe20348f17
Successfully built sumeval


In [ ]:
from sumeval.metrics.rouge import RougeCalculator
import numpy as np


rouge = RougeCalculator(stopwords=True, lang="en")
Rouge_1 = []
Rouge_2 = []
Rouge_l = []

for i in range(len(pred_str)):

    rouge_1 = rouge.rouge_n(
    summary=pred_str[i],
    references=label_str[i],
    n=1)
    
    Rouge_1.append(rouge_1)
 
 
 ## rogue_2

    rouge_2 = rouge.rouge_n(
     summary=pred_str[i],
     references=label_str[i],
    n=2)
    Rouge_2.append(rouge_2)
 ## rogue_l

    rouge_l = rouge.rouge_l(
     summary=pred_str[i],
     references=label_str[i])
    Rouge_l.append(rouge_l)

print("ROUGE-1: {}, ROUGE-2: {}, ROUGE-L: {}".format(np.round(sum(Rouge_1)/len(Rouge_1),3),
 np.round(sum(Rouge_2)/len(Rouge_2),3),
 np.round(sum(Rouge_l)/len(Rouge_l),3)
))


ROUGE-1: 0.159, ROUGE-2: 0.04, ROUGE-L: 0.128
